In [1]:
#Getting Roberta model for understanding the summary

In [2]:
import transformers
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from scipy.special import softmax
from time import time
from IPython.display import display, Markdown

In [3]:
from transformers import pipeline

2024-05-30 14:58:13.073259: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 14:58:13.073364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 14:58:13.207150: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Initialize the question-answering pipeline
# Function to find the whetehr a particular work had been done or not
def robquestion(question):
    nlp = pipeline('question-answering', model='deepset/roberta-base-squad2')

    # Define the context and the question
    text=[" Implement restart new functionality enablement for property automation in NSL Preprod/dr env", "Implement Update HPA operation enablement for property automation in NSL Preprod/dr env"]
    for part in text:
        context = ".".join([part])
    #     question = "have we implemented the update HPA operation in preprod ?"

    # Get the answer to the question
    answer = nlp(question=question, context=context)
    return answer
# print(answer)

In [8]:
print(robquestion("have we implemented the update HPA operation in preprod ?"))

{'score': 0.009687711484730244, 'start': 0, 'end': 72, 'answer': 'Implement Update HPA operation enablement for property automation in NSL'}


In [9]:
#Functions to get the issues from the JIRA
import requests
import json

# Jira credentials
JIRA_BASE_URL = '<Provide the base url for JIRA>'
ADMIN_EMAIL = '<Provide the registered Email>'
API_TOKEN = '<Provide the API Token>'

In [10]:
#Function to get all data from JIRA
def getAllJiraData():
    # Create session
    session = requests.Session()
    session.auth = (ADMIN_EMAIL, API_TOKEN)

    # Fetch all projects
    projects_url = f"{JIRA_BASE_URL}/rest/api/3/project"
    projects_response = session.get(projects_url)
    projects_data = projects_response.json()

    # Fetch all boards
    boards_url = f"{JIRA_BASE_URL}/rest/agile/1.0/board"
    boards_response = session.get(boards_url)
    boards_data = boards_response.json()

    # Fetch all fields
    fields_url = f"{JIRA_BASE_URL}/rest/api/3/field"
    fields_response = session.get(fields_url)
    fields_data = fields_response.json()

    # Fetch all issue types
    issue_types_url = f"{JIRA_BASE_URL}/rest/api/3/issuetype"
    issue_types_response = session.get(issue_types_url)
    issue_types_data = issue_types_response.json()

    # Fetch all users
    users_url = f"{JIRA_BASE_URL}/rest/api/3/users/search"
    users_response = session.get(users_url)
    users_data = users_response.json()

    # Prepare data dictionary
    jira_data = {
        'projects': projects_data,
        'boards': boards_data,
        'fields': fields_data,
        'issue_types': issue_types_data,
        'users': users_data
    }

    print(projects_data)
    # Convert JSON data to a Python object
    # data = json.loads(projects_data)

    # # Convert the Python object to a DataFrame
    # df = pd.DataFrame(data)

    # # Print the DataFrame
    # print(df)
    # Save data to JSON file
    with open('jira_data.json', 'w') as f:
        json.dump(jira_data, f, indent=4)
    !pwd

    print("Data saved to jira_data.json")


In [11]:
#Function to get Jira issues based on the query provided
def getJiraDataByQuery(query):
    def fetch_jira_query_results(jql_query):
        try:
            # Create session
            session = requests.Session()
            session.auth = (ADMIN_EMAIL, API_TOKEN)

            # Jira search API endpoint
            search_url = f"{JIRA_BASE_URL}/rest/api/3/search"
            print(search_url)

            # Set the parameters for the query
            params = {
                'jql': jql_query,
                'maxResults': 1000,  # You can adjust this number based on your needs
            }
            # params = {
            # 'jql': jql_query,
            # 'maxResults': 1000,  # Adjust this number based on your needs
            # 'fields': 'summary,description',  # Replace 'customfield_10000' with your acceptance criteria field id
            # }

            # Fetch the query results
            response = session.get(search_url, params=params, verify=False)
            response.raise_for_status()  # Raise an error for bad responses
            query_results = response.json()
            issues = response.json()['issues']
            
            text_dict = []
            for issue in issues:
                key = issue['fields']
                issuetype = key['issuetype']
                description = issuetype['description']
                text_dict.append(description)

            return [query_results,text_dict]

        except requests.exceptions.RequestException as e:
            print(f"Error fetching Jira query results: {e}")
            return [None,None]

    def save_to_json(data, filename):
        try:
            with open(filename, 'w') as f:
                json.dump(data, f, indent=4)
        except IOError as e:
            print(f"Error saving data to {filename}: {e}")

    try:
        # Input the JQL query
        # jql_query = input("Enter the JQL query: ")
        jql_query = query
        # Fetch the Jira query results
        query_results = fetch_jira_query_results(jql_query)

        if query_results[0]:
            # Save the query results to a JSON file
            save_to_json(query_results[0], 'jira_query_results.json')
            print("Query results saved to jira_query_results.json")
            return query_results[1]
        else:
            print("No data to save.")

    except Exception as e:
        print(f"An error occurred: {e}") 

In [12]:
# Model call
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return [user_message + " " + answer  + " " +  ttime, answer]


system_message = """
You are an AI assistant designed to help with scrum activities in JIRA.
Please answer with the listing of the JIRA query.
Question: {question}
Answer:
"""

In [14]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [15]:
def findJsonData(message):
    system_message = """
    You are an AI assistant designed to help with scrum activities in JIRA.
    Please answer with the listing of the JIRA query.
    Question: {question}
    Answer:
    """
    response = query_model(
        system_message,
        user_message = message,
        temperature = 0.1, 
        max_length = 256)
    query = response[1].split(":")[-1]
    query = query.replace('"', "'")
    query = query.replace('`', "")
    text_data = getJiraDataByQuery(query)
    return text_data
# display(Markdown(colorize_text(response[0])))


In [16]:
def prompt(message,system_message=""):
    if len(system_message)==0:
        system_message = """
        You are an AI assistant designed to answer simple questions.
        Please restrict your answer to the exact question asked.
        """
    response = query_model(
        system_message,
        user_message = message,
        temperature = 0.1, 
        max_length = 256)
    display(Markdown(colorize_text(response[0])))

In [17]:
def createTemplate():
    title = input("Enter the Title and issue for the Template")
    system_message = """
    You are an AI assistant designed to help with scrum activities in JIRA.
    Please answer with the listing of the JIRA issues.
    Question: {question}
    Answer:
    """
    title = "Please create a template with description and acceptance criteria in JIRA for the "+title
    prompt(title, system_message)

In [32]:
def workStatus(question):
    context = findJsonData("please find the stories with completed status")
    template = ",".join(context)
    template = "context is : " + template + "please find the answer to provide question from the question:"+ question
    system_message = """
    You are an AI assistant designed to help with reading the provided data and answer the question from the provided data.
    Please answer with the listing of the data provided and answer the question.
    Question: {question}
    Answer:
    """
    prompt(template,system_message)

In [25]:
findJsonData("please find the stories assigned to user Saksham Dubey in project avengers")

https://pathakrahul45y.atlassian.net/rest/api/3/search


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pathakrahul45y.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Query results saved to jira_query_results.json


['Stories track functionality or features expressed as user goals.']

In [26]:
createTemplate()

Enter the Title and issue for the Template Story for code merge of toggle feature in preprod environment




**<font color='red'>Question:</font>** Please create a template with description and acceptance criteria in JIRA for the Story for code merge of toggle feature in preprod environment 

**<font color='green'>Answer:</font>** Here is a template for a JIRA story with description and acceptance criteria for the code merge of the toggle feature in the preprod environment:

**Title:** Merge Toggle Feature Code into Preprod Environment

**Description:**

As part of the toggle feature development, we need to merge the code changes into the preprod environment. This will ensure that the feature is properly tested and validated before being deployed to production.

**Acceptance Criteria:**

1. The code changes for the toggle feature are successfully merged into the preprod environment.
2. The preprod environment is updated to reflect the latest code changes.
3. The toggle feature is properly configured and enabled in the preprod environment.
4. The feature is thoroughly tested in the preprod environment to ensure it functions as expected.
5. The preprod environment is validated to ensure it is identical to the production environment.

**Estimated Time:** [Insert estimated time]

**Priority:** [Insert priority]

**Labels:** [Insert labels, e.g. "toggle feature", "preprod environment", "code merge"]

**Assignee:** [Insert assignee]

**Reporter:** [Insert reporter]

Please note that this is just a template and you may need to adjust it according to your specific requirements and JIRA configuration. 

**<font color='magenta'>Total time:</font>** 18.09 sec.

In [29]:
prompt("can you tell me the steps to create a new board in JIRA ?")

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104



**<font color='red'>Question:</font>** can you tell me the steps to create a new board in JIRA ? 

**<font color='green'>Answer:</font>** Here are the steps to create a new board in JIRA:

1. Log in to your JIRA account and navigate to the "Boards" section.
2. Click on the "Create board" button.
3. Choose the board type: Scrum, Kanban, or Custom.
4. Enter a name and description for your board.
5. Select the projects and issue types you want to include on the board.
6. Choose the board layout: Swimlanes, Columns, or Custom.
7. Configure the board settings, such as the workflow and issue types.
8. Click "Create" to create the new board.

That's it! Your new board is now ready to use. 

**<font color='magenta'>Total time:</font>** 10.07 sec.

In [30]:
prompt("This is the text: Implement restart new functionality enablement for property automation in NSL Preprod/dr env, Implement Update HPA operation enablement for property automation in NSL Preprod/dr env and question is have we implemented the update HPA operation in preprod ?")



**<font color='red'>Question:</font>** This is the text: Implement restart new functionality enablement for property automation in NSL Preprod/dr env, Implement Update HPA operation enablement for property automation in NSL Preprod/dr env and question is have we implemented the update HPA operation in preprod ? 

**<font color='green'>Answer:</font>** Yes, according to the text, the second task is to "Implement Update HPA operation enablement for property automation in NSL Preprod/dr env", which implies that the update HPA operation has been implemented in the preprod environment. 

**<font color='magenta'>Total time:</font>** 3.6 sec.

In [33]:
workStatus("have we completed the toggle feature in uat environment ?")

https://pathakrahul45y.atlassian.net/rest/api/3/search


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pathakrahul45y.atlassian.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Error fetching Jira query results: 400 Client Error: Bad Request for url: https://pathakrahul45y.atlassian.net/rest/api/3/search?jql=issueStatus+%3D+%27Completed%27&maxResults=1000
No data to save.


TypeError: can only join an iterable